In [3]:
import pandas as pd
import numpy as np
from ast import literal_eval
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from fuzzywuzzy import fuzz
import fuzzymatcher

In [4]:
df1 = pd.read_csv('data/mojo_pg1.csv',converters={'crew_list': literal_eval,'cast_list': literal_eval,'summary_sales': literal_eval})
df2 = pd.read_csv('data/mojo_pg2.csv',converters={'crew_list': literal_eval,'cast_list': literal_eval,'summary_sales': literal_eval})
df3 = pd.read_csv('data/mojo_pg3.csv',converters={'crew_list': literal_eval,'cast_list': literal_eval,'summary_sales': literal_eval})
df4 = pd.read_csv('data/mojo_pg4.csv',converters={'crew_list': literal_eval,'cast_list': literal_eval,'summary_sales': literal_eval})
df5 = pd.read_csv('data/mojo_pg5.csv',converters={'crew_list': literal_eval,'cast_list': literal_eval,'summary_sales': literal_eval})
df_all = pd.concat([df1,df2,df3,df4,df5], ignore_index = True)

In [5]:
#Create dummy variables for genres
genres_list = ['Action','Adventure','Animation','Biography','Comedy','Crime','Drama','Fantasy','Family','History','Horror','Musical','Mystery','Romance','Sci-Fi','Sport','Thriller','Western']

for i in genres_list:
    df_all[i] = df_all['genres'].apply(lambda x : 1 if i in x else 0)
    
df_all['genre_top10'] = np.where((df_all['Biography'] == 1) | (df_all['Crime'] == 1) | 
                             (df_all['History'] == 1) |
                             (df_all['Horror'] == 1) |
                             (df_all['Musical'] == 1) |
                             (df_all['Mystery'] == 1) |
                             (df_all['Sport'] == 1) |
                             (df_all['Western'] == 1), 0, 1)

df_all['genre_top3'] = np.where((df_all['Action'] == 1) |
                             (df_all['Adventure'] == 1) |
                             (df_all['Comedy'] == 1), 1, 0)


df_all['genre_top2'] = np.where((df_all['Action'] == 1) |
                             (df_all['Adventure'] == 1), 1, 0)

In [6]:
#Rebucket domestic distributors to group smaller distributors
df_all['domestic_distributor'] = df_all['domestic_distributor'].replace(np.nan,'Universal Pictures')
dist_counts = df_all['domestic_distributor'].value_counts()
other_dist = list(dist_counts[dist_counts <= 50].index)

df_all['domestic_dist_processed'] = df_all['domestic_distributor'].replace(other_dist, 'Other')
df_all['domestic_dist_processed'].value_counts()


df_all['dd_top5'] = np.where((df_all['domestic_dist_processed'] == 'Other') | (df_all['domestic_dist_processed'] == 'Sony Pictures Entertainment (SPE)') | 
                             (df_all['domestic_dist_processed'] == 'Paramount Pictures') |
                             (df_all['domestic_dist_processed'] == 'Universal Pictures'), 0, 1)

In [7]:
domestic_distributors_list = df_all['domestic_dist_processed'].unique().tolist()

In [8]:
# Clean movie titles

# Clean Spider-Man
df_all.loc[(df_all['movie_title'] == 'Spider') & (df_all['earliest_release'] == '2002-05-03'), ['movie_title']] = 'Spider-Man'
df_all.loc[(df_all['movie_title'] == 'Spider') & (df_all['earliest_release'] == '2019-06-28'), ['movie_title']] = 'Spider-Man: Far From Home'
df_all.loc[(df_all['movie_title'] == 'Spider') & (df_all['earliest_release'] == '2004-06-30'), ['movie_title']] = 'Spider-Man 2'
df_all.loc[(df_all['movie_title'] == 'Spider') & (df_all['earliest_release'] == '2007-05-01'), ['movie_title']] = 'Spider-Man 3'
df_all.loc[(df_all['movie_title'] == 'Spider') & (df_all['earliest_release'] == '2017-07-05'), ['movie_title']] = 'Spider-Man: Homecoming'
df_all.loc[(df_all['movie_title'] == 'Spider') & (df_all['earliest_release'] == '2018-12-12'), ['movie_title']] = 'Spider-Man: Into the Spider-Verse 3D'
df_all.loc[(df_all['movie_title'] == 'The Amazing Spider') & (df_all['earliest_release'] == '2012-06-28'), ['movie_title']] = 'The Amazing Spider-Man'
df_all.loc[(df_all['movie_title'] == 'The Amazing Spider') & (df_all['earliest_release'] == '2014-04-16'), ['movie_title']] = 'The Amazing Spider-Man 2'


#Clean X-Men
df_all.loc[(df_all['movie_title'] == 'X') & (df_all['earliest_release'] == '2006-05-24'), ['movie_title']] = 'X-Men: The Last Stand'
df_all.loc[(df_all['movie_title'] == 'X') & (df_all['earliest_release'] == '2014-05-21'), ['movie_title']] = 'X-Men: Days of Future Past'
df_all.loc[(df_all['movie_title'] == 'X') & (df_all['earliest_release'] == '2009-04-29'), ['movie_title']] = 'X-Men Origins: Wolvering'
df_all.loc[(df_all['movie_title'] == 'X') & (df_all['earliest_release'] == '2000-07-13'), ['movie_title']] = 'X-Men: Apocalypse'
df_all.loc[(df_all['movie_title'] == 'X') & (df_all['earliest_release'] == '2016-05-18'), ['movie_title']] = 'X-Men: The Last Stand'
df_all.loc[(df_all['movie_title'] == 'X') & (df_all['earliest_release'] == '2011-06-01'), ['movie_title']] = 'X-Men: First Class'
df_all.loc[(df_all['movie_title'] == 'X2: X') & (df_all['earliest_release'] == '2003-04-30'), ['movie_title']] = 'X-Men 2'

#Clean G-Force
df_all.loc[(df_all['movie_title'] == 'G') & (df_all['earliest_release'] == '2009-07-23'), ['movie_title']] = 'G-Force'
df_all.loc[(df_all['movie_title'] == 'WALL·E') & (df_all['earliest_release'] == '2008-06-26'), ['movie_title']] = 'WALL-E'
df_all.loc[(df_all['movie_title'] == 'Les Misérables') & (df_all['earliest_release'] == '2012-12-19'), ['movie_title']] = 'Les Miserables'
df_all.loc[(df_all['movie_title'] == 'Pokémon Detective Pikachu') & (df_all['earliest_release'] == '2019-05-03'), ['movie_title']] = 'Pokemon Detective Pikachu'


In [9]:
#Fill missing runtime
df_all.loc[(df_all['movie_title'] == 'Justice League') & (df_all['earliest_release'] == '2017-10-17'), ['runtime_minutes']] = 120
df_all.loc[(df_all['movie_title'] == 'Mr. & Mrs. Smith') & (df_all['earliest_release'] == '2005-06-08'), ['runtime_minutes']] = 126
df_all.loc[(df_all['movie_title'] == 'Crazy Rich Asians') & (df_all['earliest_release'] == '2018-08-15'), ['runtime_minutes']] = 121
df_all.loc[(df_all['movie_title'] == 'Home Alone 2: Lost in New York') & (df_all['earliest_release'] == '1992-11-20'), ['runtime_minutes']] = 121
df_all.loc[(df_all['movie_title'] == 'Mad Max: Fury Road') & (df_all['earliest_release'] == '2015-05-07'), ['runtime_minutes']] = 120
df_all.loc[(df_all['movie_title'] == 'Slumdog Millionaire') & (df_all['earliest_release'] == '2008-11-12'), ['runtime_minutes']] = 123
df_all.loc[(df_all['movie_title'] == 'Argo') & (df_all['earliest_release'] == '2012-10-11'), ['runtime_minutes']] = 130
df_all.loc[(df_all['movie_title'] == 'Crouching Tiger, Hidden Dragon') & (df_all['earliest_release'] == '2000-12-08'), ['runtime_minutes']] = 120
df_all.loc[(df_all['movie_title'] == 'The Wolf of Wall Street') & (df_all['earliest_release'] == '2013-12-25'), ['runtime_minutes']] = 180
df_all.loc[(df_all['movie_title'] == 'Spy') & (df_all['earliest_release'] == '2015-05-21'), ['runtime_minutes']] = 130
df_all.loc[(df_all['movie_title'] == 'Independence Day: Resurgence') & (df_all['earliest_release'] == '2016-06-22'), ['runtime_minutes']] = 129
df_all.loc[(df_all['movie_title'] == 'Collateral') & (df_all['earliest_release'] == '2004-08-05'), ['runtime_minutes']] = 120
df_all.loc[(df_all['movie_title'] == 'The Social Network') & (df_all['earliest_release'] == '2010-10-01'), ['runtime_minutes']] = 121
df_all.loc[(df_all['movie_title'] == 'Space Station 3D') & (df_all['earliest_release'] == '2002-04-19'), ['runtime_minutes']] = 47
df_all.loc[(df_all['movie_title'] == 'The Muppets') & (df_all['earliest_release'] == '2011-11-23'), ['runtime_minutes']] = 120
df_all.loc[(df_all['movie_title'] == 'Eight Below') & (df_all['earliest_release'] == '2006-02-17'), ['runtime_minutes']] = 120

In [10]:
# Import and clean budget data for merging
df_budget = pd.read_csv('data/budget-all.csv')
df_budget.rename(columns = {'title': 'movie_title'},inplace=True)

In [11]:
left_on = ['movie_title']
right_on = ['movie_title']

#Fuzzy merge tables to match on movie title
merged_df = fuzzymatcher.fuzzy_left_join(df_all,
                                            df_budget,
                                            left_on,
                                            right_on,
                                            left_id_col='movie_title',
                                            right_id_col='movie_title')

In [12]:
merged_df.shape

(1001, 49)

In [13]:
#Replace values of budget column based on accuracy of match
merged_df['budget_left'] = np.where((merged_df['best_match_score'] > 0) & (merged_df['budget_left'].isnull()), 
                                    merged_df['budget_right'], merged_df['budget_left'])

In [14]:
#Rename columns
merged_df.rename(columns = {'movie_title_left': 'movie_title', 'budget_left':'budget'},inplace=True)
merged_df.reset_index(drop= True)

,best_match_score,__id_left,__id_right,movie_title,domestic_distributor,budget,domestic_opening_sales,earliest_release,rating,runtime_minutes,...,dd_top5,Unnamed: 0,Unnamed: 0.1,link_stub,rank_all_movies,release,movie_title_right,budget_right,domestic_gross,worldwide_gross
0,0.255865,Star Wars: Episode VII,Star Wars Ep. VII: The Force Awakens,Star Wars: Episode VII,Walt Disney Studios Motion Pictures,245000000.0,247966675.0,2015-12-16,PG-13,138.0,...,1,3.0,16-Dec-15,/box-office-chart/daily/2015/12/16,4.0,16-Dec-15,Star Wars Ep. VII: The Force Awakens,306000000.0,"$936,662,225","$2,065,478,084"
1,0.484933,Avengers: Endgame,Avengers: Endgame,Avengers: Endgame,Walt Disney Studios Motion Pictures,356000000.0,357115007.0,2019-04-24,PG-13,181.0,...,1,0.0,23-Apr-19,/box-office-chart/daily/2019/04/23,1.0,23-Apr-19,Avengers: Endgame,400000000.0,"$858,373,000","$2,797,800,564"
2,0.197412,Avatar,Avatar,Avatar,Twentieth Century Fox,237000000.0,77025481.0,2009-12-16,PG-13,162.0,...,1,21.0,17-Dec-09,/box-office-chart/daily/2009/12/17,22.0,17-Dec-09,Avatar,237000000.0,"$760,507,625","$2,788,701,337"
3,0.491031,Black Panther,Black Panther,Black Panther,Walt Disney Studios Motion Pictures,200000000.0,202003951.0,2018-02-13,PG-13,134.0,...,1,43.0,13-Feb-18,/box-office-chart/daily/2018/02/13,44.0,13-Feb-18,Black Panther,200000000.0,"$700,059,566","$1,336,494,321"
4,0.707680,Avengers: Infinity War,Avengers: Infinity War,Avengers: Infinity War,Walt Disney Studios Motion Pictures,300000000.0,257698183.0,2018-04-25,PG-13,149.0,...,1,4.0,25-Apr-18,/box-office-chart/daily/2018/04/25,5.0,25-Apr-18,Avengers: Infinity War,300000000.0,"$678,815,482","$2,044,540,523"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,NaN,Footloose,None,Footloose,Paramount Pictures,8200000.0,8556935.0,1984-02-17,NaN,107.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,0.044297,Dear John,John Carter,Dear John,Screen Gems,25000000.0,30468614.0,2010-02-05,PG-13,108.0,...,0,10.0,7-Mar-12,/box-office-chart/daily/2012/03/07,11.0,7-Mar-12,John Carter,263700000.0,"$73,058,679","$282,778,100"
998,0.649050,Men in Black: International,Men in Black: International,Men in Black: International,Sony Pictures Entertainment (SPE),110000000.0,30035838.0,2019-06-12,PG-13,114.0,...,0,50.0,12-Jun-19,/box-office-chart/daily/2019/06/12,351.0,12-Jun-19,Men in Black: International,110000000.0,"$80,001,807","$253,020,464"
999,-0.136109,A Star Is Born,Solo: A Star Wars Story,A Star Is Born,Warner Bros.,NaN,NaN,1976-12-19,NaN,139.0,...,1,9.0,23-May-18,/box-office-chart/daily/2018/05/23,10.0,23-May-18,Solo: A Star Wars Story,275000000.0,"$213,767,512","$393,151,347"


In [15]:
#Drop unnecessary columns
merged_df.drop(columns=['best_match_score','__id_left', '__id_right','Unnamed: 0.1','movie_title_right','budget_right','domestic_gross','worldwide_gross','release', 'rank_all_movies', 'link_stub'],inplace=True)

In [16]:
#Change date string to datetime
merged_df['release_date'] = pd.to_datetime(merged_df['earliest_release'])

In [17]:
# Drop years before 2000 for more complete data / more fair comparison
at_2000 = merged_df[(merged_df['release_date'].dt.year >= 2000)]

In [18]:
# Fill remaining empty budget values
for index in at_2000.index:
    if at_2000.loc[index,'movie_title']=='Over the Hedge':
        at_2000.loc[index,'budget'] = 80000000
    elif at_2000.loc[index,'movie_title']=='Marley & Me':
        at_2000.loc[index,'budget'] = 60000000
    elif at_2000.loc[index,'movie_title']=='Traffic':
        at_2000.loc[index,'budget'] = 48000000
    elif at_2000.loc[index,'movie_title']=='Hairspray':
        at_2000.loc[index,'budget'] = 75000000
    elif at_2000.loc[index,'movie_title']=='How to Lose a Guy in 10 Days':
        at_2000.loc[index,'budget'] = 50000000
    elif at_2000.loc[index,'movie_title']=='Dreamgirls':
        at_2000.loc[index,'budget'] = 75000000
    elif at_2000.loc[index,'movie_title']=='Gnomeo & Juliet':
        at_2000.loc[index,'budget'] = 36000000
    elif at_2000.loc[index,'movie_title']=='Meet the Robinsons':
        at_2000.loc[index,'budget'] = 150000000
    elif at_2000.loc[index,'movie_title']=='Beverly Hills Chihuahua':
        at_2000.loc[index,'budget'] = 20000000
    elif at_2000.loc[index,'movie_title']=="He's Just Not That Into You":
        at_2000.loc[index,'budget'] = 40000000
    elif at_2000.loc[index,'movie_title']=='Space Station 3D':
        at_2000.loc[index,'budget'] = 1000000
    elif at_2000.loc[index,'movie_title']=='John Wick: Chapter 2':
        at_2000.loc[index,'budget'] = 40000000
    elif at_2000.loc[index,'movie_title']=='The Imitation Game':
        at_2000.loc[index,'budget'] = 14000000
    elif at_2000.loc[index,'movie_title']=='Save the Last Dance':
        at_2000.loc[index,'budget'] = 13000000
    elif at_2000.loc[index,'movie_title']=='The Game Plan':
        at_2000.loc[index,'budget'] = 22000000
    elif at_2000.loc[index,'movie_title']=='Madea Goes to Jail':
        at_2000.loc[index,'budget'] = 17500000
    elif at_2000.loc[index,'movie_title']=='Flightplan':
        at_2000.loc[index,'budget'] = 55000000
    elif at_2000.loc[index,'movie_title']=='Brother Bear':
        at_2000.loc[index,'budget'] = 46000000
    elif at_2000.loc[index,'movie_title']=='The Santa Clause 3: The Escape Clause':
        at_2000.loc[index,'budget'] = 12000000
    elif at_2000.loc[index,'movie_title']=='Scooby':
        at_2000.loc[index,'budget'] = 84000000
    elif at_2000.loc[index,'movie_title']=='Smallfoot':
        at_2000.loc[index,'budget'] = 80000000
    elif at_2000.loc[index,'movie_title']=='The Descendants':
        at_2000.loc[index,'budget'] = 35000000
    elif at_2000.loc[index,'movie_title']=='Cheaper by the Dozen 2':
        at_2000.loc[index,'budget'] = 60000000
    elif at_2000.loc[index,'movie_title']=='Bridge to Terabithia':
        at_2000.loc[index,'budget'] = 20000000


/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [19]:
# Input missing value for get out
for index in at_2000.index:
    if at_2000.loc[index,'movie_title']=='Get Out':
        at_2000.loc[index,'domestic_opening_sales'] = 34000000

In [20]:
# Change dollar values to int
at_2000['budget'] = at_2000['budget'].astype(int)
at_2000['domestic_opening_sales'] = at_2000['domestic_opening_sales'].astype(int)
at_2000.reset_index(drop=True)

<ipython-input-20-902ebc41e218>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  at_2000['budget'] = at_2000['budget'].astype(int)
<ipython-input-20-902ebc41e218>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  at_2000['domestic_opening_sales'] = at_2000['domestic_opening_sales'].astype(int)


,movie_title,domestic_distributor,budget,domestic_opening_sales,earliest_release,rating,runtime_minutes,genres,summary_sales,crew_list,...,Sport,Thriller,Western,genre_top10,genre_top3,genre_top2,domestic_dist_processed,dd_top5,Unnamed: 0,release_date
0,Star Wars: Episode VII,Walt Disney Studios Motion Pictures,245000000,247966675,2015-12-16,PG-13,138.0,"['Action', 'Adventure', 'Sci-Fi']","[[Domestic, 936662225], [International, 113179...","[J.J. Abrams, Lawrence Kasdan, J.J. Abrams, Mi...",...,0,0,0,1,1,1,Walt Disney Studios Motion Pictures,1,3.0,2015-12-16
1,Avengers: Endgame,Walt Disney Studios Motion Pictures,356000000,357115007,2019-04-24,PG-13,181.0,"['Action', 'Adventure', 'Drama', 'Sci-Fi']","[[Domestic, 858373000], [International, 193942...","[Anthony Russo, Joe Russo, Christopher Markus,...",...,0,0,0,1,1,1,Walt Disney Studios Motion Pictures,1,0.0,2019-04-24
2,Avatar,Twentieth Century Fox,237000000,77025481,2009-12-16,PG-13,162.0,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']","[[Domestic, 760507625], [International, 202993...","[James Cameron, James Cameron, James Cameron, ...",...,0,0,0,1,1,1,Twentieth Century Fox,1,21.0,2009-12-16
3,Black Panther,Walt Disney Studios Motion Pictures,200000000,202003951,2018-02-13,PG-13,134.0,"['Action', 'Adventure', 'Sci-Fi']","[[Domestic, 700426566], [International, 647171...","[Ryan Coogler, Ryan Coogler, Joe Robert Cole, ...",...,0,0,0,1,1,1,Walt Disney Studios Motion Pictures,1,43.0,2018-02-13
4,Avengers: Infinity War,Walt Disney Studios Motion Pictures,300000000,257698183,2018-04-25,PG-13,149.0,"['Action', 'Adventure', 'Sci-Fi']","[[Domestic, 678815482], [International, 136954...","[Anthony Russo, Joe Russo, Christopher Markus,...",...,0,0,0,1,1,1,Walt Disney Studios Motion Pictures,1,4.0,2018-04-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,Jack Reacher,Paramount Pictures,60000000,15210156,2012-12-20,PG-13,130.0,"['Action', 'Thriller']","[[Domestic, 80070736], [International, 1382698...","[Christopher McQuarrie, Lee Child, Christopher...",...,0,1,0,1,1,1,Paramount Pictures,0,73.0,2012-12-20
719,Cloverfield,Paramount Pictures,25000000,40058229,2008-01-17,PG-13,85.0,"['Action', 'Horror', 'Sci-Fi', 'Thriller']","[[Domestic, 80048433], [International, 9234574...","[Matt Reeves, Drew Goddard, J.J. Abrams, Bryan...",...,0,1,0,0,1,1,Paramount Pictures,0,NaN,2008-01-17
720,Dear John,Screen Gems,25000000,30468614,2010-02-05,PG-13,108.0,"['Drama', 'Romance', 'War']","[[Domestic, 80014842], [International, 3496982...","[Lasse Hallström, Jamie Linden, Nicholas Spark...",...,0,0,0,1,0,0,Other,0,10.0,2010-02-05
721,Men in Black: International,Sony Pictures Entertainment (SPE),110000000,30035838,2019-06-12,PG-13,114.0,"['Action', 'Adventure', 'Comedy', 'Sci-Fi']","[[Domestic, 80001807], [International, 1738888...","[F. Gary Gray, Matt Holloway, Art Marcum, Lowe...",...,0,0,0,1,1,1,Sony Pictures Entertainment (SPE),0,50.0,2019-06-12


In [21]:
# Import data to identify if the movie is a sequel
df_sequel = pd.read_csv('data/final-data-sequel.csv')
df_sequel.reset_index(drop=True)

,movie_title,year,is_sequel
0,Star Wars: Episode VII,12/16/15,1
1,Avengers: Endgame,4/24/19,1
2,Avatar,12/16/09,0
3,Black Panther,2/13/18,1
4,Avengers: Infinity War,4/25/18,1
...,...,...,...
718,Jack Reacher,12/20/12,0
719,Cloverfield,1/17/08,0
720,Dear John,2/5/10,0
721,Men in Black: International,6/12/19,1


In [22]:
# Merge two dataframes
master_df = at_2000.merge(df_sequel, how='left')

#Drop unnecessary columns
master_df.drop(columns=['Unnamed: 0','year'],inplace=True)


In [23]:
#Drop duplicated rows
master_df.drop_duplicates(subset=master_df.columns.difference(['crew_list','cast_list','summary_sales']),keep='last',inplace = True)
#df.drop_duplicates()

In [24]:
master_df

,movie_title,domestic_distributor,budget,domestic_opening_sales,earliest_release,rating,runtime_minutes,genres,summary_sales,crew_list,...,Sport,Thriller,Western,genre_top10,genre_top3,genre_top2,domestic_dist_processed,dd_top5,release_date,is_sequel
0,Star Wars: Episode VII,Walt Disney Studios Motion Pictures,245000000,247966675,2015-12-16,PG-13,138.0,"['Action', 'Adventure', 'Sci-Fi']","[[Domestic, 936662225], [International, 113179...","[J.J. Abrams, Lawrence Kasdan, J.J. Abrams, Mi...",...,0,0,0,1,1,1,Walt Disney Studios Motion Pictures,1,2015-12-16,1
1,Avengers: Endgame,Walt Disney Studios Motion Pictures,356000000,357115007,2019-04-24,PG-13,181.0,"['Action', 'Adventure', 'Drama', 'Sci-Fi']","[[Domestic, 858373000], [International, 193942...","[Anthony Russo, Joe Russo, Christopher Markus,...",...,0,0,0,1,1,1,Walt Disney Studios Motion Pictures,1,2019-04-24,1
2,Avatar,Twentieth Century Fox,237000000,77025481,2009-12-16,PG-13,162.0,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']","[[Domestic, 760507625], [International, 202993...","[James Cameron, James Cameron, James Cameron, ...",...,0,0,0,1,1,1,Twentieth Century Fox,1,2009-12-16,0
3,Black Panther,Walt Disney Studios Motion Pictures,200000000,202003951,2018-02-13,PG-13,134.0,"['Action', 'Adventure', 'Sci-Fi']","[[Domestic, 700426566], [International, 647171...","[Ryan Coogler, Ryan Coogler, Joe Robert Cole, ...",...,0,0,0,1,1,1,Walt Disney Studios Motion Pictures,1,2018-02-13,1
4,Avengers: Infinity War,Walt Disney Studios Motion Pictures,300000000,257698183,2018-04-25,PG-13,149.0,"['Action', 'Adventure', 'Sci-Fi']","[[Domestic, 678815482], [International, 136954...","[Anthony Russo, Joe Russo, Christopher Markus,...",...,0,0,0,1,1,1,Walt Disney Studios Motion Pictures,1,2018-04-25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,Jack Reacher,Paramount Pictures,60000000,15210156,2012-12-20,PG-13,130.0,"['Action', 'Thriller']","[[Domestic, 80070736], [International, 1382698...","[Christopher McQuarrie, Lee Child, Christopher...",...,0,1,0,1,1,1,Paramount Pictures,0,2012-12-20,0
739,Cloverfield,Paramount Pictures,25000000,40058229,2008-01-17,PG-13,85.0,"['Action', 'Horror', 'Sci-Fi', 'Thriller']","[[Domestic, 80048433], [International, 9234574...","[Matt Reeves, Drew Goddard, J.J. Abrams, Bryan...",...,0,1,0,0,1,1,Paramount Pictures,0,2008-01-17,0
740,Dear John,Screen Gems,25000000,30468614,2010-02-05,PG-13,108.0,"['Drama', 'Romance', 'War']","[[Domestic, 80014842], [International, 3496982...","[Lasse Hallström, Jamie Linden, Nicholas Spark...",...,0,0,0,1,0,0,Other,0,2010-02-05,0
741,Men in Black: International,Sony Pictures Entertainment (SPE),110000000,30035838,2019-06-12,PG-13,114.0,"['Action', 'Adventure', 'Comedy', 'Sci-Fi']","[[Domestic, 80001807], [International, 1738888...","[F. Gary Gray, Matt Holloway, Art Marcum, Lowe...",...,0,0,0,1,1,1,Sony Pictures Entertainment (SPE),0,2019-06-12,1


In [25]:
master_df.loc[master_df['movie_title'] == 'Toy Story 3', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'Finding Nemo', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'The Incredibles', ['rating']] = 'PG'
master_df.loc[master_df['movie_title'] == 'Cars', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'WALL-E', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'The Polar Express', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'Fast & Furious Presents: Hobbs & Shaw', ['rating']] = 'PG-13'
master_df.loc[master_df['movie_title'] == 'Horton Hears a Who!', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'The Santa Clause 2', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'Chicken Run', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'The Princess and the Frog', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'Gnomeo & Juliet', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'Legally Blonde', ['rating']] = 'PG-13'
master_df.loc[master_df['movie_title'] == 'The Princess Diaries 2: Royal Engagement', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'Space Station 3D', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == "The Emperor's New Groove", ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'Brother Bear', ['rating']] = 'G'
master_df.loc[master_df['movie_title'] == 'Jimmy Neutron: Boy Genius', ['rating']] = 'G'

In [26]:
master_df['month'] = pd.DatetimeIndex(master_df['release_date']).month

In [27]:
master_df['director'] = master_df['crew_list'].apply(lambda x: x[0])

In [28]:
director_list = master_df['director'].unique().tolist()

In [29]:
director_df = master_df[['movie_title','budget','domestic_opening_sales','director']]

In [30]:
# Based on top 15% of directors contributing ~45% of total domestic revenue for the sample
top50_director = ['Anthony Russo','J.J. Abrams','David Yates','Christopher Nolan','Michael Bay',
                 'Zack Snyder', 'Francis Lawrence', 'Peter Jackson', 'Sam Raimi', 'Bryan Singer', 
                  'Joss Whedon', 'Jon Favreau','Gore Verbinski', 'Brad Bird', 'James Wan', 'Todd Phillips',
                  'Peyton Reed', 'Tim Burton', 'Justin Lin', 'Bill Condon', 'Ridley Scott', 'Steven Spielberg',
                  'Andrew Adamson','Andrew Stanton', 'Ron Howard', 'M. Night Shyamalan', 'Brett Ratner',
                  'Gareth Edwards', 'Rian Johnson', 'James Gunn', 'Roland Emmerich', 'Ryan Coogler', 'Tim Story',
                  'Chris Renaud','Pete Docter', 'Gary Ross', 'Andy Muschietti', 'Peter Segal', 'Chris Columbus',
                  'Jon Watts', 'Colin Trevorrow', 'F. Gary Gray', 'Paul Greengrass', 'Shawn Levy', 'Carlos Saldanha',
                  'Eric Darnell','James Mangold', 'George Lucas', 'Kyle Balda', 'David Leitch']

In [31]:
master_df['top50_d'] = np.where((master_df['director'] == 'Anthony Russo') |
                             (master_df['director'] == 'J.J. Abrams') | 
                             (master_df['director'] == 'David Yates') |
                             (master_df['director'] == 'Christopher Nolan') |
                             (master_df['director'] == 'Michael Bay') |
                             (master_df['director'] == 'Zack Snyder') |
                             (master_df['director'] == 'Francis Lawrence') |
                             (master_df['director'] == 'Peter Jackson') |
                             (master_df['director'] == 'Sam Raimi') | 
                             (master_df['director'] == 'Bryan Singer') |
                             (master_df['director'] == 'Joss Whedon') |
                             (master_df['director'] == 'Jon Favreau') |
                             (master_df['director'] == 'Gore Verbinski') |
                             (master_df['director'] == 'Brad Bird') |
                             (master_df['director'] == 'James Wan') |
                             (master_df['director'] == 'Todd Phillips') |
                             (master_df['director'] == 'Peyton Reed') |
                             (master_df['director'] == 'Tim Burton') |
                             (master_df['director'] == 'Justin Lin') |
                             (master_df['director'] == 'Bill Condon') |
                             (master_df['director'] == 'Ridley Scott') |
                             (master_df['director'] == 'Steven Spielberg') |
                             (master_df['director'] == 'Andrew Adamson') |
                             (master_df['director'] == 'Andrew Stanton') |
                             (master_df['director'] == 'Ron Howard') |
                             (master_df['director'] == 'M. Night Shyamalan') |
                             (master_df['director'] == 'Brett Ratner') |
                             (master_df['director'] == 'Gareth Edwards') |
                             (master_df['director'] == 'Rian Johnson') |
                             (master_df['director'] == 'James Gunn') |
                             (master_df['director'] == 'Roland Emmerich') |
                             (master_df['director'] == 'Ryan Coogler') |
                             (master_df['director'] == 'Tim Story') |
                             (master_df['director'] == 'Chris Renaud') |
                             (master_df['director'] == 'Pete Doctor') |
                             (master_df['director'] == 'Gary Ross') |
                             (master_df['director'] == 'Andy Muschietti') |
                             (master_df['director'] == 'Peter Segal') |
                             (master_df['director'] == 'Chris Columbus') |
                             (master_df['director'] == 'Jon Watts') |
                             (master_df['director'] == 'Colin Trevorrow') |
                             (master_df['director'] == 'F. Gary Gray') |
                             (master_df['director'] == 'Paul Greengrass') |
                             (master_df['director'] == 'Shawn Levy') |
                             (master_df['director'] == 'Carlos Saldanha') |
                             (master_df['director'] == 'Eric Darnell') |
                             (master_df['director'] == 'James Mangold') |
                             (master_df['director'] == 'George Lucas') |
                             (master_df['director'] == 'Kyle Balda') |
                             (master_df['director'] == 'David Leitch'), 1, 0)

In [32]:
# Correct sequel dummy
master_df.loc[master_df['movie_title'] == 'Terminator 3: Rise of the Machines', ['is_sequel']] = '1'

In [33]:
# Based on the Numbers list of actors who add most value (revenue) to films
bankable_list = ['Tom Cruise','Robert Downey Jr.','Kathleen Kennedy','Zack Snyder','Will Smith',
                 'Kevin Feige','Denzel Washington','Clint Eastwood','Alan Silvestri','Steven Spielberg',
                 'Jon Favreau','Leonardo DiCaprio','John Lasseter','John Williams','Bradley Cooper',
                 'Michael Keaton','Emma Watson','Ian McKellen', 'Ben Affleck', 'Kenneth Branagh',
                'Matt Damon', 'Julia Roberts','Chiwetel Ejiofor','Tom Hanks','Jim Carrey',
                'Samuel L. Jackson','Dwayne Johnson', 'George Clooney', 'Scarlett Johansson',
                'Bruce Berman', 'Vin Diesel','Chris Evans','Harrison Ford','David Heyman','Johnny Depp',
                'Djimon Hounsou','Mark Ruffalo','Robin Wright','Giovanni Ribisi','Daniel Craig',
                'Brad Pitt','Paul Bettany','Morgan Freeman','Ryan Reynolds','Gal Gadot','Angelina Jolie',
                'Jason Statham','Thomas Tull','Tommy Lee Jones','James Newton Howard']

for i in bankable_list:
    master_df[i] = master_df['cast_list'].apply(lambda x : 1 if i in x else 0)

In [34]:
master_df['bankable_count'] =  master_df[[ 'Tom Cruise','Robert Downey Jr.','Kathleen Kennedy','Zack Snyder','Will Smith',
                 'Kevin Feige','Denzel Washington','Clint Eastwood','Alan Silvestri','Steven Spielberg',
                 'Jon Favreau','Leonardo DiCaprio','John Lasseter','John Williams','Bradley Cooper',
                 'Michael Keaton','Emma Watson','Ian McKellen', 'Ben Affleck', 'Kenneth Branagh',
                'Matt Damon', 'Julia Roberts','Chiwetel Ejiofor','Tom Hanks','Jim Carrey',
                'Samuel L. Jackson','Dwayne Johnson', 'George Clooney', 'Scarlett Johansson',
                'Bruce Berman', 'Vin Diesel','Chris Evans','Harrison Ford','David Heyman','Johnny Depp',
                'Djimon Hounsou','Mark Ruffalo','Robin Wright','Giovanni Ribisi','Daniel Craig',
                'Brad Pitt','Paul Bettany','Morgan Freeman','Ryan Reynolds','Gal Gadot','Angelina Jolie',
                'Jason Statham','Thomas Tull','Tommy Lee Jones','James Newton Howard']].sum(axis=1)

In [35]:
# Sales currently stored as list - create new column for each
master_df['domestic_sales'] = master_df['summary_sales'].apply(lambda x: [int(y[1]) for y in x if y[0] == 'Domestic'])
master_df['international_sales'] = master_df['summary_sales'].apply(lambda x: [int(y[1]) for y in x if y[0] == 'International'])
master_df['worldwide_sales'] = master_df['summary_sales'].apply(lambda x: [int(y[1]) for y in x if y[0] == 'Worldwide'])

In [36]:
master_df[['movie_title','summary_sales','domestic_sales']]

,movie_title,summary_sales,domestic_sales
0,Star Wars: Episode VII,"[[Domestic, 936662225], [International, 113179...",[936662225]
1,Avengers: Endgame,"[[Domestic, 858373000], [International, 193942...",[858373000]
2,Avatar,"[[Domestic, 760507625], [International, 202993...",[760507625]
3,Black Panther,"[[Domestic, 700426566], [International, 647171...",[700426566]
4,Avengers: Infinity War,"[[Domestic, 678815482], [International, 136954...",[678815482]
...,...,...,...
738,Jack Reacher,"[[Domestic, 80070736], [International, 1382698...",[80070736]
739,Cloverfield,"[[Domestic, 80048433], [International, 9234574...",[80048433]
740,Dear John,"[[Domestic, 80014842], [International, 3496982...",[80014842]
741,Men in Black: International,"[[Domestic, 80001807], [International, 1738888...",[80001807]


In [37]:
master_df['domestic_gross'] = master_df['domestic_sales'].apply(pd.to_numeric)
master_df['domestic_gross'] = master_df.domestic_sales.apply(lambda x: x[0])

In [38]:
master_df[['movie_title','summary_sales','domestic_sales','domestic_gross']]

,movie_title,summary_sales,domestic_sales,domestic_gross
0,Star Wars: Episode VII,"[[Domestic, 936662225], [International, 113179...",[936662225],936662225
1,Avengers: Endgame,"[[Domestic, 858373000], [International, 193942...",[858373000],858373000
2,Avatar,"[[Domestic, 760507625], [International, 202993...",[760507625],760507625
3,Black Panther,"[[Domestic, 700426566], [International, 647171...",[700426566],700426566
4,Avengers: Infinity War,"[[Domestic, 678815482], [International, 136954...",[678815482],678815482
...,...,...,...,...
738,Jack Reacher,"[[Domestic, 80070736], [International, 1382698...",[80070736],80070736
739,Cloverfield,"[[Domestic, 80048433], [International, 9234574...",[80048433],80048433
740,Dear John,"[[Domestic, 80014842], [International, 3496982...",[80014842],80014842
741,Men in Black: International,"[[Domestic, 80001807], [International, 1738888...",[80001807],80001807


In [39]:
master_df.to_csv('data/master_data_with_sequel_director.csv')